# การสร้าง Model สำหรับ Text Classification โดยใช้ Bag-of-word Feature กับ Standard Feedforward Neural Network

In [97]:
pip install thai_tokenizer

In [98]:
import csv
import numpy as np
import deepcut
from keras.models import Model
from keras.layers import Input, Dense, Bidirectional, LSTM, Embedding, Dropout, Flatten, concatenate
from tensorflow.keras.utils import to_categorical, set_random_seed
import matplotlib.pyplot as plt
from random import shuffle

set_random_seed(99)

**Read raw data form input.txt and ans.txt**

In [117]:
with open('dataset/input.txt', 'r', encoding='utf-8-sig') as input_file:
    input_data = [line.strip().split('::')[1] for line in input_file]

with open('dataset/ans.txt', 'r', encoding='utf-8-sig') as ans_file:
    ans_data = [line.strip().split('::')[1] for line in ans_file]
    
combined_data = []

for input_line, ans_line in zip(input_data, ans_data):
    combined_line = [ans_line, input_line]
    combined_data.append(combined_line)

# Shuffle the input data
shuffle(combined_data)
for combined_line in combined_data:
    print(combined_line)

['T,T,T', 'ในตานี้การ์ดในมือเน่า รอตาหน้าจะได้การ์ดดีๆ สรุปตาหน้าก็เน่าเหมือนเดิม']
['E', 'ตาเธอสวยเหมือนลูกแก้ว']
['E', 'ผู้คนมักไปทำตาสองชั้นที่ประเทศเกาหลี']
['P', 'ตากินยาหลังอาหารยัง']
['E', 'ซุปเปอร์แมนปล่อยเรเซอร์ออกมาจากตาเพื่อโกน']
['P,E', 'น้าจะพาคุณตาไปตรวจตาวันไหนหรอครับ']
['E,E', 'ภายนอกเขาอาจจะดูเหมือนคนปกติแต่เขาตาบอดตาใสน่ะ']
['E,E', 'ตาวิเศษจงบอกข้า ใครที่มีดวงตางามเริดสุดในดินแดนนี้']
['T,T', 'เอกสุ่มกาชาเกมส์ Genshin ไป 10 ตาพบว่าเกลือทุกตาเลย']
['E', 'ตาตี่แล้วมันผิดตรงไหน']
['T', 'ใครๆก็เล่นเกมเกิน1ตา']
['T', 'ตานี้ลงมอนไม่ได้เลย']
['T,T', 'ฉันเล่นตานี้ตาท้ายนะ']
['E,P', 'ด่าว่าฉันตาไม่ถึงเลยหรอ งั้นก็ไปร้องขอตานั่นละกันว่าขอส่งช้ากว่ากำหนด']
['T', 'เมื่อไหร่ตานี้จะจบสักที หิวข้าวแล้ว']
['P', 'อยู่ดีๆตาของฉันก็หายไป']
['E,E,P', 'ฉันอยากมีตาเหมือนกับนกเหยี่ยว เพราะสายตาของนกเหยี่ยวจะมองได้ไกล คุณตาของฉันเคยบอกไว้แบบนั้น']
['P', 'ตาเดินไปตลาด']
['P', 'ตาทำอาหาร']
['P', 'คุณตาที่อยู่อเมริกาโดยวัยรุ่นที่นั้นทำร้ายจนเสียชีวิต']
['E', 'ในสายตาของฉันนั้นมีแต่พี่ยองบิน']
[

**Create dataset. Extract labels and sentences.**

In [100]:
labels = [d[0] for d in combined_data]
sentences = [d[1] for d in combined_data]
test_labels = labels[3855:]

print(labels)
print(sentences)
print(test_labels)

['T,T', 'P,E', 'P', 'T', 'E,E', 'E', 'P,P', 'P,E', 'E', 'E', 'P,T', 'E', 'E', 'T,T', 'T', 'P,T', 'T,T,T', 'E', 'P,P', 'E', 'P', 'E', 'P,P', 'E', 'E', 'E,E', 'E', 'E', 'E', 'E', 'E', 'E', 'T', 'E', 'P', 'P', 'P,E', 'T', 'P', 'E', 'E', 'P', 'E,E', 'E,P', 'P', 'P', 'P', 'P,E,P', 'P,P', 'E', 'P,E', 'P,E', 'T,E,P', 'P', 'E,E', 'E,E', 'P', 'E', 'E', 'E', 'P', 'T,P,E', 'T', 'P,P,P', 'P', 'E', 'P', 'T', 'E,E,E', 'P,T', 'E,P', 'P', 'P', 'E', 'E', 'P,E', 'P', 'T,T', 'E', 'P,P', 'E', 'T', 'E', 'T,E', 'T,P', 'P', 'E', 'P', 'E', 'P,P', 'T,T', 'T', 'E', 'E', 'E', 'E', 'P', 'E,P', 'P,P', 'E', 'P', 'P,P', 'T', 'E,E', 'E,E', 'E', 'E,E', 'E,E', 'P,P,P', 'P,T', 'E', 'P', 'T', 'E,E', 'P', 'E', 'E', 'E', 'E', 'E', 'E', 'T', 'T', 'P,P', 'E', 'P', 'P', 'E,E', 'P', 'E', 'T,P', 'P,P', 'P', 'T,E', 'P,E', 'E', 'E', 'E', 'E', 'T', 'T,P', 'P,E', 'T', 'T', 'E', 'T', 'T', 'P', 'E,T', 'P', 'P', 'T', 'P', 'T', 'E', 'P', 'E', 'P', 'E', 'P,E', 'T', 'E', 'E,P', 'P', 'E', 'T', 'P', 'P,P', 'E,P,P', 'P', 'P,E', 'E', 'P', 'E

**Tokenize each sentence into a list of words.**

In [101]:
from pythainlp.tokenize import word_tokenize
from thai_tokenizer import Tokenizer

tokenizer = Tokenizer()

tokenized_sentences = [
    [word for word in word_tokenize(sentence, engine="newmm", keep_whitespace=False)]
    for sentence in sentences
]

for sentence_tokens in tokenized_sentences:
    print(sentence_tokens)


['ตา', 'นี้', 'ก็', 'เข้า', 'เกม', 'ไม่', 'ได้', 'ตา', 'หน้า', 'ก็', 'ไม่', 'ได้', 'เล่น']
['ตา', 'ของ', 'แม่', 'สายตายาว']
['นางสาว', 'มุทิตา', 'ได้', 'รางวัล', 'จาก', 'ตา', 'เพราะ', 'เรียน', 'ได้', 'ที่หนึ่ง']
['ไม่', 'มี', 'ตา', 'ไหน', 'ที่', 'เธอ', 'ไม่', 'โยน', 'เกม']
['หมอ', 'ตา', 'ตรวจ', 'ดวงตา', 'ของ', 'ฉัน']
['การ', 'นอนตะแคง', 'เล่น', 'โทรศัพท์', 'มีโอกาส', 'ทำให้', 'สายตาเอียง']
['เธอ', 'ดู', 'สิ', 'ตา', 'คน', 'นี้', 'นี่แหละ', 'ที่', 'ถูก', 'ทิ้ง', 'ไป', 'ตอน', 'เด็ก', 'ทำไม', 'ตา', 'คน', 'นี้', 'ถึง', 'มีชีวิต', 'ที่', 'ดีขึ้น', 'ขนาด', 'นี้', 'นะ']
['ตาแก่', 'สอง', 'คน', 'กำลัง', 'นั่ง', 'จ้อง', 'ตา', 'กัน']
['vel', "'", 'koz', 'เป็น', 'สัตว์ประหลาด', 'จาก', 'Void', 'ที่', 'มี', 'ดวงตา', 'ขนาดใหญ่', 'อยู่', 'ตรงกลาง']
['ตา', 'ของ', 'เขา', 'เป็น', 'สีน้ำตาล']
['คุณตา', 'และ', 'คุณยาย', 'ตอนนี้', 'แก่', 'มาก', 'แล้ว', 'เลย', 'ทำให้', 'ดวงตา', 'ฝ้าฟาง']
['สายตา', 'เธอ', 'หลอก', 'ฉัน', 'ไม่', 'ได้']
['ฉัน', 'จึง', 'เลิกใช้', 'ที่', 'ปัด', 'ขน', 'ตา']
['ตา', 'นี้', 'บอด', 'ตา'

**Find all possible words (vocab).**

In [102]:
vocab = set([w for s in tokenized_sentences for w in s])
print('Vocab size = '+str(len(vocab)))
print(vocab)
vocab_size = int(len(vocab))
print(vocab_size)

Vocab size = 4608
{'พระ', 'ชิด', 'ศัตรู', 'ทำบุญ', 'หายนะ', 'วันหนึ่ง', 'สมรัก', 'หลับตา', 'ป่่วย', 'โหด', 'สำนัก', 'ดอกไม้ไฟ', 'ปลั๊ก', 'นอกบ้าน', 'อร', 'แขก', 'วันข้างหน้า', 'เข้ารอบ', 'อาการหนัก', 'อารมณ์', 'เต่า', 'อธิบาย', 'เสียชีวิต', 'ซ่า', 'มืดมน', 'คำย่อ', 'ห่วยแตก', 'ต่างจังหวัด', 'ตานี', 'อาย', 'น้ำส้ม', 'ให้อาหาร', 'กะ', 'ของ', 'อ่าน', 'อัสดง', 'ร่ำรวย', 'โชค', 'พอใจ', 'ทั้งนั้น', 'ไหว้', 'ตาขวาง', 'วางสาย', 'นักบิน', 'เฉียบคม', 'แพง', 'ขาย', 'มีความรู้', 'คอ', 'สิงโต', 'ที่มา', 'ไหล่', 'ฆาตกร', 'คอน', 'กลับบ้าน', 'เป็นธรรมดา', 'ได้ตัว', 'รึเปล่า', 'โกง', 'อุ้ย', 'มังกร', 'จับจ้อง', 'ป.', 'ย้อย', 'เงียบ ๆ', 'ทีไร', 'ปกป้อง', 'วิเศษ', 'แก้วตา', 'กำเริบ', 'เม้นต์', 'ชะมัด', 'ประหม่า', 'เหงื่อ', 'ตาเหลือก', 'รื้อ', 'ไว', 'เฉี่ยว', 'พายุหมุน', 'ขึ้นรถ', 'ชีวิต', 'น้ำยา', 'ตากลม', 'จา', 'มีค่า', 'หาก', 'ปะการัง', 'เลือดตาแทบกระเด็น', 'ปลูกถ่าย', 'ตลก', 'ไปมา', 'ฝรั่ง', 'พิส', 'ซุ่ม', 'มะ', 'เสียเถิด', 'โยม', 'จัก', 'Sir', 'ได้เวลา', 'เริ่มใหม่', 'พักตา', 'เหล่านั้น', 'ต่างกัน', 

In [103]:
labels = [label.split(',') for label in labels]
labels = [[value for value in label if value != ''] for label in labels]
label_mapping = {'E': 0, 'e': 0, 'P': 1, 'p': 1, 'T': 2, 't': 2}
labels = [[label_mapping[value] for value in label] for label in labels]

print(labels)

[[2, 2], [1, 0], [1], [2], [0, 0], [0], [1, 1], [1, 0], [0], [0], [1, 2], [0], [0], [2, 2], [2], [1, 2], [2, 2, 2], [0], [1, 1], [0], [1], [0], [1, 1], [0], [0], [0, 0], [0], [0], [0], [0], [0], [0], [2], [0], [1], [1], [1, 0], [2], [1], [0], [0], [1], [0, 0], [0, 1], [1], [1], [1], [1, 0, 1], [1, 1], [0], [1, 0], [1, 0], [2, 0, 1], [1], [0, 0], [0, 0], [1], [0], [0], [0], [1], [2, 1, 0], [2], [1, 1, 1], [1], [0], [1], [2], [0, 0, 0], [1, 2], [0, 1], [1], [1], [0], [0], [1, 0], [1], [2, 2], [0], [1, 1], [0], [2], [0], [2, 0], [2, 1], [1], [0], [1], [0], [1, 1], [2, 2], [2], [0], [0], [0], [0], [1], [0, 1], [1, 1], [0], [1], [1, 1], [2], [0, 0], [0, 0], [0], [0, 0], [0, 0], [1, 1, 1], [1, 2], [0], [1], [2], [0, 0], [1], [0], [0], [0], [0], [0], [0], [2], [2], [1, 1], [0], [1], [1], [0, 0], [1], [0], [2, 1], [1, 1], [1], [2, 0], [1, 0], [0], [0], [0], [0], [2], [2, 1], [1, 0], [2], [2], [0], [2], [2], [1], [0, 2], [1], [1], [2], [1], [2], [0], [1], [0], [1], [0], [1, 0], [2], [0], [0, 1]

In [104]:
# Word Vector train
max_sentence_length = max([len(s) for s in tokenized_sentences])
print(max_sentence_length)
word_vector_length = 300
from pythainlp import word_vector
wv = word_vector.WordVector(model_name="thai2fit_wv").get_model() # load thai2fit_wv from pythainlp

word_vectors_train = np.zeros(
    (3855, max_sentence_length, word_vector_length)
)
sample_count = 0
for s in tokenized_sentences[:3855]:
    word_count = 0
    for w in s:
        try:
            word_vectors_train[
                sample_count, max_sentence_length - word_count - 1, :
            ] = wv[w]
            word_count = word_count + 1
        except:
            pass
    sample_count = sample_count + 1

y_train = labels[:3855]
# print(word_vectors_train)

34


**Extract bag-of-word feature for each sentence**

In [105]:
# Word Vector train
max_sentence_length = max([len(s) for s in tokenized_sentences])
print(max_sentence_length)
word_vector_length = 300
from pythainlp import word_vector
wv = word_vector.WordVector(model_name="thai2fit_wv").get_model() # load thai2fit_wv from pythainlp

word_vectors_test = np.zeros(
    (len(tokenized_sentences[3855:]), max_sentence_length, word_vector_length)
)
sample_count = 0
for s in tokenized_sentences[3855:]:
    word_count = 0
    for w in s:
        try:
            word_vectors_test[
                sample_count, max_sentence_length - word_count - 1, :
            ] = wv[w]
            word_count = word_count + 1
        except:
            pass
    sample_count = sample_count + 1

y_test = labels[3855:]
# print(word_vectors_test)

34


In [106]:
from keras.utils import to_categorical
import numpy as np

# Define a function to prepare y_train_last and y_test_last
def prepare_targets(y_data):
    y1, y2, y3 = [], [], []
    
    for i in y_data:
        if len(i) >= 1:
            y1.append(i[0])
        else:
            y1.append(3)
        
        if len(i) >= 2:
            y2.append(i[1])
        else:
            y2.append(3)
        
        if len(i) == 3:
            y3.append(i[2])
        else:
            y3.append(3)

    y1 = to_categorical(y1, num_classes=4)
    y2 = to_categorical(y2, num_classes=4)
    y3 = to_categorical(y3, num_classes=4)

    return [y1, y2, y3]

y_train_last = prepare_targets(y_train)
y_test_last = prepare_targets(y_test)

print(vocab_size, word_vector_length)
# print(word_vectors_test)


4608 300


In [107]:
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
import tensorflow as tf

num_classes = 4  
input_layer = Input(shape=(max_sentence_length, word_vector_length))

rnn1 = Bidirectional(LSTM(128, return_sequences=True))(input_layer)
rnn1 = BatchNormalization()(rnn1)
rnn1 = Bidirectional(LSTM(128))(rnn1)
rnn1 = BatchNormalization()(rnn1)
rnn1 = Dropout(0.2)(rnn1)

shared_hidden_layer = Dense(64, activation="relu")(rnn1)
shared_hidden_layer = BatchNormalization()(shared_hidden_layer)
shared_hidden_layer = Dropout(0.3)(shared_hidden_layer)

output1 = Dense(num_classes, activation="softmax", name="output1")(shared_hidden_layer)
output2 = Dense(num_classes, activation="softmax", name="output2")(shared_hidden_layer)
output3 = Dense(num_classes, activation="softmax", name="output3")(shared_hidden_layer)

# Create the model
model = Model(inputs=input_layer, outputs=[output1, output2, output3])

losses = {
    "output1": "categorical_crossentropy",
    "output2": "categorical_crossentropy",
    "output3": "categorical_crossentropy",
}

loss_weights = {
    "output1": 1.0,
    "output2": 1.0,
    "output3": 1.0,
}

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=losses, loss_weights=loss_weights, metrics=["accuracy"])

model.summary()


Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 34, 300)]    0           []                               
                                                                                                  
 bidirectional_20 (Bidirectiona  (None, 34, 256)     439296      ['input_11[0][0]']               
 l)                                                                                               
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 34, 256)     1024        ['bidirectional_20[0][0]']       
 rmalization)                                                                                     
                                                                                           

In [108]:
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_output3_accuracy', save_best_only=True, verbose=1)

history = model.fit(word_vectors_train, y_train_last, epochs=600, batch_size=50, validation_split=0.2, callbacks=[model_checkpoint])

Epoch 1/600
62/62 [==============================] - ETA: 0s - loss: 4.4228 - output1_loss: 1.3403 - output2_loss: 1.5363 - output3_loss: 1.5462 - output1_accuracy: 0.5240 - output2_accuracy: 0.4293 - output3_accuracy: 0.4219
Epoch 1: val_output3_accuracy improved from -inf to 0.82490, saving model to best_model.h5
62/62 [==============================] - 35s 404ms/step - loss: 4.4228 - output1_loss: 1.3403 - output2_loss: 1.5363 - output3_loss: 1.5462 - output1_accuracy: 0.5240 - output2_accuracy: 0.4293 - output3_accuracy: 0.4219 - val_loss: 3.4513 - val_output1_loss: 1.1491 - val_output2_loss: 1.2588 - val_output3_loss: 1.0435 - val_output1_accuracy: 0.6278 - val_output2_accuracy: 0.4436 - val_output3_accuracy: 0.8249
Epoch 2/600
62/62 [==============================] - ETA: 0s - loss: 2.8604 - output1_loss: 0.8799 - output2_loss: 1.0389 - output3_loss: 0.9416 - output1_accuracy: 0.6984 - output2_accuracy: 0.6329 - output3_accuracy: 0.7046
Epoch 2: val_output3_accuracy improved from

In [109]:
model.save('model.h5')

In [116]:
from keras.models import Model, load_model
import numpy as np

model = load_model('model.h5')

class_mapping = {0: "E", 1: "P", 2: "T"}
y_pred = model.predict(word_vectors_test)

y_pred_labels = []
for i in range(len(y_pred[0])):
    output1_argmax = y_pred[0][i].argmax()
    output1_label = (
        class_mapping[output1_argmax] if output1_argmax in class_mapping else ""
    )

    output2_argmax = y_pred[1][i].argmax()
    output2_label = (
        class_mapping[output2_argmax] if output2_argmax in class_mapping else ""
    )

    output3_argmax = y_pred[2][i].argmax()
    output3_label = (
        class_mapping[output3_argmax] if output3_argmax in class_mapping else ""
    )

    # Combine the predicted labels
    predicted_labels = [
        label for label in [output1_label, output2_label, output3_label] if label != ""
    ]
    predicted_labels_str = ",".join(predicted_labels)

    y_pred_labels.append(f"{i + 1}::{predicted_labels_str}")

# with open("ans_test_1.txt", "w") as f:
#     f.write("\n".join(y_pred_labels))

y_pred_labels = [label.split("::")[1] for label in y_pred_labels]

print(y_pred_labels)
print(test_labels)

true = 0
false = 0
for i in test_labels:
    if i in y_pred_labels:
        true = true + 1
    else:
        false = false + 1


percent = true / (true + false) * 100
print("Test Length: " + str(len(test_labels)))
print("True: " + str(true))
print("False: " + str(false))
print(format(percent, '.2f'), "%")

31/31 [==============================] - 6s 124ms/step
['T,P,P', 'P,E', 'E,E', 'E', 'E', 'E', 'P,E', 'P', 'T', 'E', 'E', 'T', 'P,E', 'T', 'P', 'P', 'P', 'E', 'T', 'E', 'P,P', 'T', 'T', 'E', 'T', 'E,P', 'E,E', 'P,E', 'E', 'T', 'T,E,P', 'E,E,T', 'P', 'E,E,E', 'E', 'E', 'E', 'P', 'P', 'T', 'T', 'E', 'E,E', 'E', 'P', 'T,T', 'P', 'P', 'E', 'E', 'P', 'P', 'E', 'P', 'T,T', 'P', 'E', 'P', 'P', 'T,T,T', 'T', 'E', 'P,T,T', 'T,T,P', 'E', 'P', 'T,E', 'E', 'E,E', 'P,E', 'E', 'P,E', 'E', 'E', 'T', 'P', 'T,T', 'P', 'P', 'P', 'P', 'P', 'E', 'T,T', 'E', 'P', 'E', 'P', 'E', 'P', 'E', 'E', 'E', 'T', 'T,T', 'E', 'P', 'E', 'E,E', 'P,T,E', 'E,E', 'E', 'E', 'T', 'T,T', 'P,E', 'E', 'E,P,E', 'P', 'P,P', 'E', 'T,T', 'T', 'E', 'E', 'E', 'E', 'T', 'P', 'E', 'P', 'T', 'E', 'P,E', 'T,T', 'E', 'P,P', 'E', 'T,T', 'E,P', 'P', 'T', 'P', 'E', 'P,P', 'P', 'T', 'P,E', 'P,E', 'E', 'T', 'P', 'T,P,E', 'P', 'T', 'E', 'P', 'P', 'E,E', 'P', 'P', 'T,T,P', 'P,E', 'E,E', 'P', 'P', 'P', 'E', 'E', 'P,P', 'P', 'E', 'P', 'P', 'E', 'P'